In [3]:
#!/usr/bin/env python3

import json
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from typing import Dict, List, Tuple
import warnings
warnings.filterwarnings('ignore')

class DeFiCreditScorer:
    def __init__(self):
        self.feature_weights = {
            'reliability_score': 0.25,      # Payment history and consistency
            'utilization_score': 0.20,      # Debt-to-collateral ratios
            'longevity_score': 0.15,        # Account age and activity duration
            'diversity_score': 0.10,        # Asset and action diversity
            'liquidation_risk_score': 0.15, # Liquidation history and risk
            'bot_behavior_score': 0.10,     # Anti-bot detection
            'volume_consistency_score': 0.05 # Transaction volume patterns
        }

    def load_data(self, json_file_path: str) -> pd.DataFrame:
        """Load and preprocess transaction data from JSON file"""
        with open(json_file_path, 'r') as f:
            data = json.load(f)

        # Convert to DataFrame
        df = pd.DataFrame(data)

        # Print available columns for debugging
        print(f"Available columns: {list(df.columns)}")
        print(f"Sample record structure detected")

        # Extract nested actionData fields
        if 'actionData' in df.columns:
            print("Extracting nested actionData fields...")

            # Extract actionData into separate columns
            action_data_df = pd.json_normalize(df['actionData'])

            # Combine with main dataframe
            df = pd.concat([df.drop('actionData', axis=1), action_data_df], axis=1)

            print(f"Updated columns after extraction: {list(df.columns)}")

        # Map the specific fields from your data structure
        field_mapping = {
            'wallet': 'userWallet',  # userWallet -> wallet
            'asset': 'assetSymbol',  # assetSymbol -> asset
            'amount': 'amount',      # amount -> amount (already correct)
            'action': 'action',      # action -> action (already correct)
            'timestamp': 'timestamp' # timestamp -> timestamp (already correct)
        }

        # Apply field mapping
        for standard_name, source_field in field_mapping.items():
            if source_field in df.columns:
                if standard_name != source_field:
                    df[standard_name] = df[source_field]
                print(f"Mapped {source_field} -> {standard_name}")
            else:
                print(f"Warning: {source_field} not found in data")

        # Validate required fields exist
        required_fields = ['wallet', 'action', 'asset', 'amount', 'timestamp']
        missing_fields = [field for field in required_fields if field not in df.columns]

        if missing_fields:
            print(f"Error: Missing required fields: {missing_fields}")
            print("Available fields:", list(df.columns))
            return pd.DataFrame()

        # Convert data types
        try:
            # Convert timestamp
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
        except Exception as e:
            print(f"Warning: Could not convert timestamp: {e}")

        # Convert amount to numeric (handle string amounts like "2000000000")
        df['amount'] = pd.to_numeric(df['amount'], errors='coerce')

        # Handle USDC decimals (USDC has 6 decimals, so divide by 1e6)
        # Detect if amounts seem to be in raw token units
        if df['amount'].median() > 1000000:  # Likely raw token units
            print("Converting amounts from raw token units...")
            # Apply decimal conversion based on common token decimals
            df['amount'] = df['amount'] / 1e6  # Assuming USDC-like tokens (6 decimals)

        # Remove rows with invalid data
        original_count = len(df)
        df = df.dropna(subset=['wallet', 'action', 'timestamp', 'amount'])
        df = df[df['amount'] > 0]  # Remove zero/negative amounts

        print(f"Loaded {len(df)} valid transactions (filtered from {original_count})")
        print(f"Unique wallets: {df['wallet'].nunique()}")
        print(f"Actions: {df['action'].unique()}")
        print(f"Assets: {df['asset'].unique()}")
        print(f"Amount range: ${df['amount'].min():.2f} - ${df['amount'].max():,.2f}")

        return df

    def engineer_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """Engineer features for each wallet"""
        wallet_features = []

        for wallet in df['wallet'].unique():
            wallet_data = df[df['wallet'] == wallet].copy()
            wallet_data = wallet_data.sort_values('timestamp')

            features = self._calculate_wallet_features(wallet_data)
            features['wallet'] = wallet
            wallet_features.append(features)

        return pd.DataFrame(wallet_features)

    def _calculate_wallet_features(self, wallet_data: pd.DataFrame) -> Dict:
        """Calculate comprehensive features for a single wallet"""
        features = {}

        # Basic transaction metrics
        features.update(self._basic_transaction_features(wallet_data))

        # Reliability and payment behavior
        features.update(self._reliability_features(wallet_data))

        # Utilization and risk management
        features.update(self._utilization_features(wallet_data))

        # Behavioral patterns
        features.update(self._behavioral_features(wallet_data))

        # Liquidation and risk indicators
        features.update(self._liquidation_risk_features(wallet_data))

        # Bot detection features
        features.update(self._bot_detection_features(wallet_data))

        return features

    def _basic_transaction_features(self, wallet_data: pd.DataFrame) -> Dict:
        """Calculate basic transaction statistics"""
        return {
            'total_transactions': len(wallet_data),
            'unique_assets': wallet_data['asset'].nunique(),
            'unique_actions': wallet_data['action'].nunique(),
            'total_volume': wallet_data['amount'].sum(),
            'avg_transaction_size': wallet_data['amount'].mean(),
            'account_age_days': (wallet_data['timestamp'].max() - wallet_data['timestamp'].min()).days + 1,
            'transactions_per_day': len(wallet_data) / max(1, (wallet_data['timestamp'].max() - wallet_data['timestamp'].min()).days + 1)
        }

    def _reliability_features(self, wallet_data: pd.DataFrame) -> Dict:
        """Calculate reliability and payment behavior features"""
        # Repayment behavior
        borrows = wallet_data[wallet_data['action'] == 'borrow']
        repays = wallet_data[wallet_data['action'] == 'repay']

        borrow_volume = borrows['amount'].sum() if len(borrows) > 0 else 0
        repay_volume = repays['amount'].sum() if len(repays) > 0 else 0

        # Transaction frequency consistency
        daily_txns = wallet_data.groupby(wallet_data['timestamp'].dt.date).size()
        frequency_consistency = 1 - (daily_txns.std() / max(daily_txns.mean(), 1))

        return {
            'repayment_ratio': repay_volume / max(borrow_volume, 1),
            'borrow_count': len(borrows),
            'repay_count': len(repays),
            'frequency_consistency': max(0, frequency_consistency),
            'avg_days_between_transactions': self._calculate_avg_time_between_txns(wallet_data)
        }

    def _utilization_features(self, wallet_data: pd.DataFrame) -> Dict:
        """Calculate utilization and risk management features"""
        deposits = wallet_data[wallet_data['action'] == 'deposit']
        borrows = wallet_data[wallet_data['action'] == 'borrow']

        total_deposits = deposits['amount'].sum() if len(deposits) > 0 else 0
        total_borrows = borrows['amount'].sum() if len(borrows) > 0 else 0

        # Calculate utilization ratio
        utilization_ratio = total_borrows / max(total_deposits, 1)

        # Healthy utilization is typically below 0.8 (80%)
        healthy_utilization_score = max(0, 1 - max(0, utilization_ratio - 0.8) * 5)

        return {
            'total_deposits': total_deposits,
            'total_borrows': total_borrows,
            'utilization_ratio': utilization_ratio,
            'healthy_utilization_score': healthy_utilization_score,
            'deposit_count': len(deposits),
            'deposit_to_borrow_ratio': len(deposits) / max(len(borrows), 1)
        }

    def _behavioral_features(self, wallet_data: pd.DataFrame) -> Dict:
        """Calculate behavioral pattern features"""
        # Time-based patterns
        wallet_data['hour'] = wallet_data['timestamp'].dt.hour
        wallet_data['day_of_week'] = wallet_data['timestamp'].dt.dayofweek

        # Diversity scores
        asset_diversity = wallet_data['asset'].nunique() / max(len(wallet_data), 1)
        action_diversity = wallet_data['action'].nunique() / 5  # 5 possible actions

        # Transaction timing patterns
        hour_std = wallet_data['hour'].std()
        day_variety = wallet_data['day_of_week'].nunique()

        return {
            'asset_diversity_score': asset_diversity,
            'action_diversity_score': action_diversity,
            'hour_pattern_consistency': 1 - (hour_std / 12) if hour_std else 1,
            'day_variety_score': day_variety / 7,
            'weekend_activity_ratio': len(wallet_data[wallet_data['day_of_week'].isin([5, 6])]) / len(wallet_data)
        }

    def _liquidation_risk_features(self, wallet_data: pd.DataFrame) -> Dict:
        """Calculate liquidation and risk indicators"""
        liquidations = wallet_data[wallet_data['action'] == 'liquidationcall']

        # Recent activity weight (more recent = higher weight)
        recent_cutoff = wallet_data['timestamp'].max() - timedelta(days=30)
        recent_activity = wallet_data[wallet_data['timestamp'] >= recent_cutoff]
        recent_activity_ratio = len(recent_activity) / len(wallet_data)

        return {
            'liquidation_count': len(liquidations),
            'liquidation_volume': liquidations['amount'].sum() if len(liquidations) > 0 else 0,
            'was_liquidated': 1 if len(liquidations) > 0 else 0,
            'recent_activity_ratio': recent_activity_ratio,
            'risk_score': min(1, len(liquidations) * 0.3)  # Each liquidation increases risk
        }

    def _bot_detection_features(self, wallet_data: pd.DataFrame) -> Dict:
        """Calculate features to detect bot-like behavior"""
        # Transaction amount patterns
        amount_std = wallet_data['amount'].std()
        amount_mean = wallet_data['amount'].mean()
        amount_cv = amount_std / max(amount_mean, 1)  # Coefficient of variation

        # Time interval patterns
        time_diffs = wallet_data['timestamp'].diff().dropna()
        time_intervals = time_diffs.dt.total_seconds()

        # Round number bias (bots often use round numbers)
        round_amounts = wallet_data['amount'].apply(lambda x: x == round(x, 0) if pd.notna(x) else False)
        round_ratio = round_amounts.mean() if len(round_amounts) > 0 else 0

        # Regularity detection
        regularity_score = 0
        if len(time_intervals) > 1:
            interval_std = time_intervals.std()
            interval_mean = time_intervals.mean()
            regularity_score = max(0, 1 - (interval_std / max(interval_mean, 1)))

        return {
            'amount_variability': min(1, amount_cv),
            'round_number_ratio': round_ratio,
            'transaction_regularity': regularity_score,
            'bot_like_score': (round_ratio * 0.4 + regularity_score * 0.6) if regularity_score > 0.8 else 0
        }

    def _calculate_avg_time_between_txns(self, wallet_data: pd.DataFrame) -> float:
        """Calculate average time between transactions in days"""
        if len(wallet_data) < 2:
            return 0

        time_diffs = wallet_data['timestamp'].diff().dropna()
        avg_diff = time_diffs.mean()
        return avg_diff.total_seconds() / 86400  # Convert to days

    def calculate_component_scores(self, features_df: pd.DataFrame) -> pd.DataFrame:
        """Calculate component scores for each wallet"""
        scores_df = features_df.copy()

        # Reliability Score (0-1)
        scores_df['reliability_score'] = self._calculate_reliability_score(features_df)

        # Utilization Score (0-1)
        scores_df['utilization_score'] = self._calculate_utilization_score(features_df)

        # Longevity Score (0-1)
        scores_df['longevity_score'] = self._calculate_longevity_score(features_df)

        # Diversity Score (0-1)
        scores_df['diversity_score'] = self._calculate_diversity_score(features_df)

        # Liquidation Risk Score (0-1, inverted)
        scores_df['liquidation_risk_score'] = self._calculate_liquidation_risk_score(features_df)

        # Bot Behavior Score (0-1, inverted)
        scores_df['bot_behavior_score'] = self._calculate_bot_behavior_score(features_df)

        # Volume Consistency Score (0-1)
        scores_df['volume_consistency_score'] = self._calculate_volume_consistency_score(features_df)

        return scores_df

    def _calculate_reliability_score(self, df: pd.DataFrame) -> pd.Series:
        """Calculate reliability score based on repayment behavior and consistency"""
        # Repayment behavior (40%)
        repayment_component = np.clip(df['repayment_ratio'], 0, 1) * 0.4

        # Frequency consistency (30%)
        frequency_component = df['frequency_consistency'].fillna(0) * 0.3

        # Transaction regularity (30%)
        regularity_component = (1 - np.clip(df['avg_days_between_transactions'] / 30, 0, 1)) * 0.3

        return repayment_component + frequency_component + regularity_component

    def _calculate_utilization_score(self, df: pd.DataFrame) -> pd.Series:
        """Calculate utilization score based on healthy borrowing patterns"""
        return df['healthy_utilization_score'].fillna(0.5)

    def _calculate_longevity_score(self, df: pd.DataFrame) -> pd.Series:
        """Calculate longevity score based on account age and activity"""
        # Account age component
        age_score = np.clip(df['account_age_days'] / 365, 0, 1) * 0.6

        # Transaction frequency component
        freq_score = np.clip(df['transactions_per_day'], 0, 5) / 5 * 0.4

        return age_score + freq_score

    def _calculate_diversity_score(self, df: pd.DataFrame) -> pd.Series:
        """Calculate diversity score based on asset and action variety"""
        return (df['asset_diversity_score'].fillna(0) * 0.6 +
                df['action_diversity_score'].fillna(0) * 0.4)

    def _calculate_liquidation_risk_score(self, df: pd.DataFrame) -> pd.Series:
        """Calculate liquidation risk score (inverted - lower risk = higher score)"""
        # Invert risk score so higher is better
        risk_component = 1 - df['risk_score'].fillna(0)
        liquidation_penalty = 1 - (df['liquidation_count'] * 0.2)

        return np.clip(risk_component * liquidation_penalty, 0, 1)

    def _calculate_bot_behavior_score(self, df: pd.DataFrame) -> pd.Series:
        """Calculate bot behavior score (inverted - less bot-like = higher score)"""
        return 1 - df['bot_like_score'].fillna(0)

    def _calculate_volume_consistency_score(self, df: pd.DataFrame) -> pd.Series:
        """Calculate volume consistency score"""
        # Moderate variability is good, too low or too high suggests issues
        variability = df['amount_variability'].fillna(1)
        optimal_variability = 0.3

        deviation = np.abs(variability - optimal_variability)
        return np.clip(1 - deviation * 2, 0, 1)

    def calculate_final_credit_scores(self, scores_df: pd.DataFrame) -> pd.DataFrame:
        """Calculate final credit scores (0-1000) using weighted components"""
        final_scores = scores_df.copy()

        # Calculate weighted score
        weighted_score = 0
        for component, weight in self.feature_weights.items():
            if component in scores_df.columns:
                weighted_score += scores_df[component] * weight

        # Convert to 0-1000 scale
        final_scores['credit_score'] = np.clip(weighted_score * 1000, 0, 1000)

        # Add score categories
        final_scores['score_category'] = final_scores['credit_score'].apply(self._categorize_score)

        return final_scores

    def _categorize_score(self, score: float) -> str:
        """Categorize credit score into risk levels"""
        if score >= 800:
            return "Excellent (Low Risk)"
        elif score >= 700:
            return "Good (Moderate Risk)"
        elif score >= 600:
            return "Fair (Elevated Risk)"
        elif score >= 500:
            return "Poor (High Risk)"
        else:
            return "Very Poor (Very High Risk)"

    def generate_score_report(self, scores_df: pd.DataFrame) -> Dict:
        """Generate a comprehensive scoring report"""
        report = {
            'total_wallets': len(scores_df),
            'average_score': scores_df['credit_score'].mean(),
            'score_distribution': scores_df['score_category'].value_counts().to_dict(),
            'component_averages': {},
            'top_wallets': scores_df.nlargest(5, 'credit_score')[['wallet', 'credit_score', 'score_category']].to_dict('records'),
            'bottom_wallets': scores_df.nsmallest(5, 'credit_score')[['wallet', 'credit_score', 'score_category']].to_dict('records')
        }

        # Component averages
        for component in self.feature_weights.keys():
            if component in scores_df.columns:
                report['component_averages'][component] = scores_df[component].mean()

        return report

    def score_wallets(self, json_file_path: str, output_file: str = None) -> Tuple[pd.DataFrame, Dict]:
        """Main function to score wallets from JSON file"""
        print("Loading transaction data...")
        df = self.load_data(json_file_path)

        print(f"Loaded {len(df)} transactions for {df['wallet'].nunique()} unique wallets")

        print("Engineering features...")
        features_df = self.engineer_features(df)

        print("Calculating component scores...")
        scores_df = self.calculate_component_scores(features_df)

        print("Calculating final credit scores...")
        final_scores_df = self.calculate_final_credit_scores(scores_df)

        print("Generating report...")
        report = self.generate_score_report(final_scores_df)

        # Save results if output file specified
        if output_file:
            final_scores_df.to_csv(output_file, index=False)
            print(f"Results saved to {output_file}")

        return final_scores_df, report

def main():
    """Example usage"""
    # Initialize scorer
    scorer = DeFiCreditScorer()

    # Score wallets from user-wallet-transactions.json
    try:
        results_df, report = scorer.score_wallets('user-wallet-transactions.json', 'wallet_credit_scores.csv')

        # Display summary
        print("\n=== CREDIT SCORING REPORT ===")
        print(f"Total Wallets Analyzed: {report['total_wallets']}")
        print(f"Average Credit Score: {report['average_score']:.1f}")

        print("\nScore Distribution:")
        for category, count in report['score_distribution'].items():
            print(f"  {category}: {count} wallets")

        print("\nComponent Score Averages:")
        for component, avg_score in report['component_averages'].items():
            print(f"  {component}: {avg_score:.3f}")

        print("\nTop 5 Wallets:")
        for wallet_info in report['top_wallets']:
            print(f"  {wallet_info['wallet']}: {wallet_info['credit_score']:.1f} ({wallet_info['score_category']})")

        print("\nBottom 5 Wallets:")
        for wallet_info in report['bottom_wallets']:
            print(f"  {wallet_info['wallet']}: {wallet_info['credit_score']:.1f} ({wallet_info['score_category']})")

    except FileNotFoundError:
        print("Error: Please ensure 'user-wallet-transactions.json' exists in the current directory")
        print("The file should contain transaction data with fields: wallet, action, asset, amount, timestamp")

if __name__ == "__main__":
    main()

Loading transaction data...
Available columns: ['_id', 'userWallet', 'network', 'protocol', 'txHash', 'logId', 'timestamp', 'blockNumber', 'action', 'actionData', '__v', 'createdAt', 'updatedAt']
Sample record structure detected
Extracting nested actionData fields...
Updated columns after extraction: ['_id', 'userWallet', 'network', 'protocol', 'txHash', 'logId', 'timestamp', 'blockNumber', 'action', '__v', 'createdAt', 'updatedAt', 'type', 'amount', 'assetSymbol', 'assetPriceUSD', 'poolId', 'userId', 'toId', 'borrowRateMode', 'borrowRate', 'variableTokenDebt', 'stableTokenDebt', 'callerId', 'useATokens', 'repayerId', 'liquidatorId', 'collateralAmount', 'collateralAssetPriceUSD', 'principalAmount', 'borrowAssetPriceUSD', 'collateralReserveId', 'collateralReserveSymbol', 'principalReserveId', 'principalReserveSymbol']
Mapped userWallet -> wallet
Mapped assetSymbol -> asset
Mapped amount -> amount
Mapped action -> action
Mapped timestamp -> timestamp
Converting amounts from raw token uni